<a href="https://colab.research.google.com/github/lmencisoe/CDA/blob/main/taller%204/Funcion_entrenamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from pandas_profiling import ProfileReport
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
import joblib
from google.colab import drive
drive.mount('/content/drive')
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay, precision_score, recall_score, f1_score, auc
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import RandomizedSearchCV
import tensorflow as tf
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import seaborn as sns
import warnings
import re
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Data de reentrenamiento

In [ ]:
df_2 = pd.read_json('https://raw.githubusercontent.com/lmencisoe/CDA/main/taller%204/DataSet_Entrenamiento_v2.json')

Trae el mejor modelo

In [ ]:
json_modelo = joblib.load('/content/drive/MyDrive/CDA_taller4/xgb_best_hp.pkl')

Trae los hiperparámetros del mejor modelo

In [ ]:
mejor_modelo = joblib.load('/content/drive/MyDrive/CDA_taller4/pipeline_seleccionado.pkl')

Define la función con el mismo pipeline y compara AUC de los modelos para data de train y data de test

In [ ]:
def entrenar(data):
  with open("/content/drive/MyDrive/CDA_taller4/version.txt", "r") as version:
     model_version = version.read()
  data_f = data.copy()
  data_f['target'] = np.where(data_f['Churn']!= 'Yes', 1, 0)
  Y_total = data_f['target'].astype(float)
  mejor_modelo = joblib.load('/content/drive/MyDrive/CDA_taller4/pipeline_seleccionado.pkl')
  data_f['TotalCharges'] = data_f['TotalCharges'].replace("", 0)
  data_f['TotalCharges'] = data_f['TotalCharges'].astype(float)
  X_total = data_f.drop(['Churn', 'customerID','target'], axis=1)
  X_train, X_test, y_train, y_test = train_test_split(X_total, Y_total, test_size=0.2, random_state=2022)
  cat_features = X_total.select_dtypes(exclude=["float64", "int64"]).columns.to_list()
  numeric_features = X_total.select_dtypes(["float64", "int64"]).columns.to_list()

  categorical_transformer = Pipeline(
            steps=[
               #('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                ('encoder', OneHotEncoder())
            ]
        )    

  numeric_transformer = Pipeline(
            steps=[
                #('imputer', SimpleImputer(strategy='most_frequent')),
                ('scaler', StandardScaler())
            ]
        )

  preprocessor = ColumnTransformer(
            transformers=[
                ('numerical', numeric_transformer, numeric_features),
                ('categorical', categorical_transformer, cat_features)
            ]
        )
  xgb_pipeline = Pipeline(
            [
                ("preprocessor", preprocessor),
                (
                    "classifier", XGBClassifier(**json_modelo),
                ),
            ]
        )
  fit_pipe = xgb_pipeline.fit(X_train, y_train)
  resul1 = " ".join(['AUC anterior modelo train:' , mejor_modelo.score(X_train, y_train).astype(str)])
  resul2 = " ".join(['AUC anterior modelo test:' , mejor_modelo.score(X_test, y_test).astype(str)])
  resul3 = " ".join(['AUC nuevo modelo train:' , fit_pipe.score(X_train, y_train).astype(str)])
  resul4 = " ".join(['AUC nuevo modelo test:' , fit_pipe.score(X_test, y_test).astype(str)])
  joblib.dump(fit_pipe, f'/content/drive/MyDrive/CDA_taller4/pipeline_seleccionado_{model_version}.pkl')
  with open("/content/drive/MyDrive/CDA_taller4/version.txt", "w") as version:
    version.write(str(int(model_version)+1))

  return resul1, resul2, resul3, resul4  
  #return xgb_pipeline
  #return X_train


In [ ]:
entrenar(df_2)

('AUC anterior modelo train: 0.79625',
 'AUC anterior modelo test: 0.825',
 'AUC nuevo modelo train: 0.8775',
 'AUC nuevo modelo test: 0.825')